<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Modelado de Tópicos 

Este *cuaderno* trata sobre modelado de tópicos a partir de datos de texto. El objetivo del *cuaderno* es que usted obtenga una visión general del modelo de asignación latente de Dirichlet (LDA, por sus siglas en inglés). Busca tambien que sea capaz de crear e implementar este modelo en `Python` y que sea  capaz evaluar de interpretar los resultados e identificar el mejor modelo de tópicos para un determinado problema. 

**NO** es necesario editar el archivo o hacer una entrega. Sin embargo, los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar  libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.



## Introducción

El modelado de tópicos o temas es una faceta del procesamiento del lenguaje natural (NLP, por sus siglas en inglés). Como vimos anteriormente utilizar el lenguaje, textos, como datos puede ser extremadamente poderoso. En este *cuaderno* nos centraremos sobre el modelado de tópicos. Inmediatamente nos surge la pregunta ¿qué son los tópicos? Responderemos esa pregunta con un ejemplo. Habremos notado que en los días en que se llevan a cabo eventos importantes (como elecciones nacionales, desastres naturales o eventos deportivos), las publicaciones de las redes sociales tienden a centrarse en esos eventos. Las publicaciones de alguna manera reflejan los eventos del día, y lo hacen de diferentes maneras. Las publicaciones pueden tener, y tendrán, puntos de vista divergentes que pueden ser agrupados en clústeres de tópicos de alto nivel. Si tuviéramos tweets sobre la final del Mundial, los tópicos de esos tweets podrían cubrir puntos de vista divergentes, que van desde la calidad del arbitraje al comportamiento de los aficionados. En Estados Unidos, el presidente realiza un discurso anual entre mediados y finales de enero llamado Estado de la Unión. Con un número suficiente de publicaciones en las redes sociales, podríamos inferir o predecir las reacciones de alto nivel (tópicos) al discurso. Esto lo lograríamos agrupando las publicaciones usando las palabras claves contenidas en ellos. Por ejemplo la siguiente figura con un breve texto sobre ciencia de datos muestra como se pueden identificar palabras y asignarlas a tópicos.

<center>
<img src = "figs/topicos1.jpg" alt = "topicos" style = "width: 500px;"/>
</center>


En  la figura entonces se muestra como palabras como información, predicció, estadística son asignadas al tópico de modelado; mientras que computacional, produccion y escala son asignadas al tópico de ingeniería. Los modelos de tópicos son importantes porque ofrecen la misma función para los datos textuales que las estadísticas clásicas para los datos numéricos. Es decir que proporcionan un resumen significativo de los datos. 

Los modelos de tópicos entran en la categoría de aprendizaje no supervisado porque, casi siempre, no se conocen de antemano los tópicos subyacentes de los documentos. Por lo tanto, no existe una variable que guie el aprendizaje. En términos de aprendizaje no supervisado, los modelos de tópicos se pueden pensar como parte del análisis de clusters, más específicamente a K-medias. Recordemos que con K-medias, primero se establece el número de clusters y luego el modelo asigna cada uno de los datos a uno de los clústers predeterminados. Lo mismo ocurre generalmente con los modelos de tópicos. Seleccionamos el número de tópicos al inicio y luego el modelo aísla las palabras que forman esa cantidad de tópicos. Este es un excelente punto de partida para una descripción general de modelado de tópicos de alto nivel.


Los modelos de tópicos buscan encontrar patrones comunes en el texto en el sentido de que los documentos descrien tópicos similares. Es decir, estos modelos identifican los tópicos abstractos en una colección de documentos (también referidos como corpus), utilizando las palabras contenidas en los documentos.  Para ello asumen que las palabras en el mismo documento están relacionadas y usan esa suposición para definir tópicos abstractos al encontrar grupos de palabras que aparecen con frecuencia una cerca de otra.  Es decir, si una oración contiene las palabras salario, empleado, y reunión, podemos asumir que esa oración trata o que su tópico es el trabajo. .  

Este tipo de algoritmos por lo general consta de cuatro pasos:

1. Determinar el número de tópicos.
2. Identificar palabras o frases concurrentes en los documentos.
3. Buscar clusters de palabras que caracterizan el documentos.
4. Retornar un conjunto de tópicos abstractos que caracterizan el corpus.

Un aspecto clave de los modelos de tópicos es que no producen tópicos específicos de una palabra o una frase, sino conjunto de palabras, cada una de las cuales representa un tópico abstracto. Esto se debe a que los modelos de tópicos entienden la proximidad de las palabras, no el contexto. Por ejemplo en la figura siguiente, el modelo no tiene idea de lo que significan ala, elevar, piloto, equipaje, pasajer, o mosca; sólo sabe que estas palabras, generalmente, siempre que aparecen, aparecen muy próximas entre sí. Será nuestra tarea darle una interpretación (o no) a este tópico.

<center>
<img src = "figs/topicos3.jpg" alt = "topicos3" style = "width: 500px;"/>
</center>

Existen varios algoritmos de modelado de tópicos, pero quizás el más conocidos es el de Asignación latente de Dirichlet, o Latent Dirichlet Allocation (LDA) en inglés. En este *cuaderno* nos centraremos en este.

## Asignación latente de Dirichlet (LDA)

En 2003, David Blei, Andrew Ng, y Michael Jordan publicaron un artículo sobre el algoritmo del modelado de tópicos conocido como Latent Dirichlet Allocation (LDA). LDA es un modelo probabilístico generativo, esto significa que el proceso de modelado comienza con el texto y funciona como ingeniería reversa a través del proceso que suponemos que lo generó, con el fin de identificar los parámetros de interés. En este caso, son los tópicos que generaron los datos que son de interés. 

Esencialmente LDA es una técnica de clustering que puede ser aplicada a colecciones de datos discretos como los son los documentos de texto. LDA es un modelo bayesiano jerárquico de tres niveles en donde cada elemento o palabra de un texto se modela como una mezcla finita de tópicos. A su vez, cada tópico se modela como una combinación infinita de palabras. En este contexto, decimos que LDA es un modelo probabilístico generativo pues estas distribuciones resultan en una representación explícita de cada conjunto de datos o documento.

Esta técnica de aprendizaje no supervisado se diferencia de las técnicas de clustering estudiadas anteriormente porque en este caso cada observación pertenece a más de un grupo. En ese orden de ideas, se puede decir que LDA es una técnica de agrupamiento difuso (*fuzzy o soft clustering*) donde la pertenencia de un elemento a un grupo se modela como una distribución de probabilidades.

### Set-up del modelo

La intución detras del modelo es que los documentos pueden ser representados como una combinación aleatoria de los tópicos latentes, donde cada tópico está caracterizado por una distribución de probabilidad sobre las palabras.

Para poder describir como funciona el modelo necesitamos primero definir ciertos términos:

- Una *palabra* es la unidad básica de los datos discretos. Definimos una palabra como un item de un vocabulario indexado por $\{1, \cdots, V\}$. Se representan las palabras mediante vectores de base uno, es decir, un vector donde solo un elemento es 1 y el resto son 0. Así, usando superíndices para denotar componentes, la v-ésima palabra en el vocabulario se representa mediante un V-vector $w$ tal que $w^v = 1$ y $w^u = 0$ para $u\neq v$.
- Un *documento* es una secuencia de $N$ palabras denotadas por $\mathbf{w}=(w_1,w_2,\cdots,w_N)$, en donde $w_n$ es la n-ésima palabra de la secuencia. 
- Un *corpus* es una colección de $M$ documentos dentodas por $\mathbf{D}=(\mathbf{w_1},\mathbf{w_2},\cdots,\mathbf{w_m})$.


Para cada documento del corpus, LDA supone los siguientes procesos generadores de cada documento $\mathbf(w)$ en el corpus $D$:

1. Elegimos $N\sim Poisson(\xi)$., donde $N$ es el número de palabras y $\xi$ es el parámetro que controla  distribución de Poisson.
2. Elegimos $\theta\sim Dir(\alpha)$, donde $\theta$ es la distribución de tópicos.
3. Para cada palabra $w_n$ de las $N$ palabras:

    a. Escogemos un tópico $z_n\sim Multinomial(\theta)$.
    
    b. Escogemos una palabra $w_n$ de $p(w_n|z_n, \beta)$, una probabilidad condicionada en el tópico $z_n$.


Estos tres paso se repiten para cada documento en el corpus. El primer paso es elegir el número de palabras en el documento tomando muestras, de la distribución de Poisson.

Luego de seleccionar $N$, debemos generar la distribución de tópicos, únicos para cada documento. Pensemos en esto como una lista de tópicos por documento con probabilidades de que representan el monto del documento representado por cada tópico. Consideremos tres tópicos: A, B y C. Un ejemplo podría ser 100% del tópico A, 75% del tópico B y 25% del tópico C, o una infinidad de otras combinaciones.

Por último, las palabras específicas en el documento se seleccionarán a través de una declaración de probabilidad condicionado al tópico seleccionado y la distribución de palabras para ese tópico. Tener en cuenta  que los documentos no se generan realmente de esta manera, pero es un proxy razonable.

Este proceso puede considerarse como una distribución sobre distribuciones. Un documento es
seleccionado de la colección (distribución) de documentos, y se selecciona un tópico (a través de la distribución multinomial) de la distribución de probabilidad de tópicos para ese documento, generado por la distribución de Dirichlet.















Este modelo inicial cuenta con algunas simplificaciones que más adelante serán removidas. En primer lugar, la dimensionalidad $k$ de la distribución Dirichlet (y por consiguiente la dimensionalidad de la variables de los tópicos $z$) se supone fija y conocida. Segundo, las probabilidades de cada palabra son parametrizadas por una matriz $\beta$ de tamaño $k\times V$ en donde $\beta_{ij}=p(w^j = 1|z^i = 1)$, que por ahora trataremos como una cantidad fija que será estimada. Finalmente, el supuesto de $N$ distribuido Poisson no es critico ya que otras distribuciones más realistas sobre la longitud de los documentos pueden ser usadas según la necesidad del investigador. Más aún, note que $N$ es independiente del resto de variables generadoras ($\theta$ y $\mathbf{z}$) por tanto ignoraremos su aleatoriedad en el desarrollo posterior.

Una variable aleatoria $\theta$ de dimensión $k$ puede tomar valores en el $(k-1)-símplex$ (un vector de dimensión $k$ llamado $\theta$ cae en el $(k-1)-símplex$ si $\theta_i\geq0, \sum_{i=1}^k\theta_i=1$), y tiene la siguiente función de densidad en el símplex:

$$p(\theta|\alpha)=\frac{\Gamma(\sum_{i=1}^k\alpha_i)}{\prod_{i=1}^k\Gamma(\alpha_i)}\theta_1^{(\alpha_1-1)}\cdots \theta_k^{(\alpha_k-1)}$$

En donde el parámetro $\alpha$ es un vector de dimensión $k$ con componentes $a_i$ tal que $a_i\geq0$, y $\Gamma(x)$ es la función Gamma. La Dirichlet es una distribución conveniente para el símplex porque está en la familia exponencial, tiene estadísticas suficientes de dimensión finita y es conjugada a la distribución multinomial.

Antes de continuar, vale la pena profundizar un poco sobre qué es un símplex y por qué es relevante en este modelo. Podemos pensar en el LDA como una aproximación geométrica en donde el símplex es una figura con $k$ vertices equidistantes entre ellos. Un 0-símplex es un punto, un 1-símplex es un segmento de línea, un 2-símplex es un triángulo, un 3-símplex es un tetraedro y un 4-símplex es un pentácoron.

<center>
<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Simplexes.jpg/800px-Simplexes.jpg" alt = "simplex" style = "width: 500px;"/>
</center>

En el caso de LDA, los documentos dentro del corpus se distribuirían a lo largo del símplex en donde cada vértice se representa un tópico. Luego, cada documento se ubicaría más cercano a los vértices que representan los tópicos contenidos en él. Por ejemplo, supongamos que tenemos 7 documentos y tres tópicos posibles (Deporte, Ciencia y Política), podríamos representar los documentos dentro del símplex de la siguiente manera:

<center>
<img src = "data/2-simplex.png" alt = "LDA" style = "width: 500px;"/>
</center>

De esta manera podríamos ver que cada documento es una combinación de tópicos. El documento 1 sería 100% sobre Deportes, el documento 2 sería 50% sobre Deportes y 50% sobre Ciencia, el documento 3 sería 100% sobre Ciencia, etc.

Continuando con nuestra explicación del modelo. Tomando como dados los parámetros $\alpha$ y $\beta$, la distribución de probabilidad conjunta de una mezcla de tópicos $\theta$, un conjunto de $N$ temas $\mathbf{z}$ y un conjunto de $N$ palabras $\mathbf{w}$ es:

$$p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta) = p(\theta|\alpha)\prod_{n=1}^N p(z_n|\theta)p(w_n|z_n, \beta)$$

En donde $p(z_n|\theta)$ es simplemente $\theta_i$ para el único $i$ en donde $z_n^i=1$. 

A continuación se muestra una representación gráfica del modelo. El rectángulo exterior M representa los documentos y el rectángulo interior N representa la escogencia repetida de palabras ($w$) y tópicos ($z$) al interior de un documento.

<center>
<img src = "data/LDA1.png" alt = "LDA1" style = "width: 500px;"/>
</center>

De la expresión anterior podemos decir que el lado izquierdo de la ecuación ($p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta)$) corresponde a la probabilidad de que un documento $x$ aparezca en nuestro corpus. De este modo, nuestro objetivo será estimar $\alpha$ y $\beta$ de modo que se maximice la probabilidad de encontrar nuestra muestra de documentos.

La Figura anterior corresponde a una ilustración de la ecuación en cuestión. En donde $\alpha$ es una distribución Dirichlet que le asigna una combinación de tópicos a cada uno de los documentos. $\beta$ es una distribución Dirichlet que le asigna una combinación de palabras a cada uno de los tópicos. A partir de $\alpha$ obtenemos $\theta$ la cual es una distribución multinomial de donde se extraen $N$ tópicos. Luego, para cada uno de los tópicos se extrae una palabra de la distribución multinomial que sale de $\beta$.

### Inferencia variacional
El principal problema de inferencia al que nos enfrentamos a la hora de usar LDA es el cálculo de la distribución posterior de las variables escondidas dado un documento:

$$p(\theta, \mathbf{z}|\mathbf{w},\alpha, \beta)=\frac{p(\theta, \mathbf{z}, \mathbf{w}|\alpha, \beta)}{p(\mathbf{w}|\alpha, \beta)}$$

Desafortunadamente, esta distribución no se puede calcular directamente, por ende se debe aproximar numéricamente. Entre los métodos que existen para aproximarnos a la solución están la aproximación de Laplace, Aproximación variacional, Cadena de Makov Monte Carlo, etc.

En esta sección vamos a describir la intuición detrás de un algoritmo variacional simple basado en la convexidad. La idea principal detrás de este método es hacer uso de la [Desigualdad de Jensen](https://es.wikipedia.org/wiki/Desigualdad_de_Jensen) para obtener un límite inferior ajustable en la log-verosimilitud. En esencia, se considera una familia de límites inferiores indexada a un conjunto de *parámetros variacionales*. Los parámetros variacionales se eligen mediante un procedimiento de optimización que intenta encontrar el límite inferior más ajustado posible.

De forma más intuitiva podemos explicar el algoritmo de la siguiente manera:
1. Comience asignando aleatoriamente cada palabra de cada documento del corpus a uno de los temas.
2. Para cada documento y cada palabra en cada documento por separado, calcule dos proporciones:
    - La proporción de palabras en el documento que están actualmente asignadas al tema: $p(Tema|Documento)$
    - La proporción de asignaciones en todos los documentos de una palabra específica al tema: $p(Palabra|Tema)$
3. Multiplique las dos proporciones y use la proporción resultante para asignar la palabra a un nuevo tema. 
4. Repita este proceso hasta que se alcance un estado estable en el que las asignaciones de temas no cambien significativamente. Estas asignaciones luego se utilizan para estimar la mezcla de temas dentro del documento y la mezcla de palabras dentro del tema.

La lógica detrás de la inferencia variacional es que, si la distribución real es intratable, entonces se debe encontrar una distribución más simple, llamémosla distribución variacional, muy cercana a la distribución verdadera, que es manejable, para que la inferencia sea posible. En otras palabras, dado que es imposible inferir la distribución real debido a la complejidad de la distribución real, tratamos de encontrar una distribución más simple que sea una excelente aproximación de la distribución real.

En nuestro caso, el problema de inferencia se da por el acomplamiento entre los parámetros $\beta$ y $\theta$ debido a los enlaces entre $\theta$, $\mathbf{z}$ y $\mathbf{w}$. Eliminando estas aristas y los nodos $\mathbf{w}$, y dotando al modelo gráfico simplificado resultante de parámetros variacionales libres, obtenemos una familia de distribuciones de las variables latentes. Esta familia se caracteriza por la siguiente distribución variacional:

$$q(\theta, \mathbf{z} | \gamma, \phi) = q(\theta|\gamma)\prod_{n=1}^N q(z_n|\phi_n)$$

En donde $\gamma$ es el parámetro Dirichlet y los parámetros multinomiales $(\phi_1,\cdots, \phi_N)$ son los parámetros variacionales libres. A continuación se ilustra gráficamente la representación del modelo de la distribución variacional utilizada para aproximar la distribución posterior en LDA.

<center>
<img src = "data/LDA2.png" alt = "LDA2" style = "width: 300px;"/>
</center>

Para comenzar, se selecciona una familia de distribuciones (es decir, binomial, gaussiana, exponencial, etc.), $q$, condicionada a los nuevos parámetros variacionales. Los parámetros están optimizados para que la distribución original (que en realidad es la distribución posterior), y la distribución variacional sean lo más parecidas posible. La distribución variacional será lo suficientemente cercana a la distribución posterior original para ser utilizada como proxy, haciendo que cualquier inferencia hecha sobre ella sea aplicable a la distribución posterior original.

El objetivo de encontrar un límite inferior ajustado en la log-verosimilitud se traduce directamente en el siguiente problema de optimización para determinar los parámetros variacionales $\gamma$ y $\phi$:

$$(\gamma^*,\phi^*)=\argmin_{(\gamma,\phi)} D(q(\theta, \mathbf{z}|\gamma,\phi)\ ||\ p(\theta, \mathbf{z}|\mathbf{w}, \alpha, \beta))$$

Hay una gran colección de distribuciones variacionales potenciales que se pueden usar como una aproximación para la distribución posterior. Se selecciona una distribución variacional inicial de la colección, que actúa como punto de partida para un proceso de optimización que iterativamente se acerca más y más a la distribución óptima. Los parámetros óptimos son los parámetros de la distribución que mejor se aproximan al posterior. La similitud de las dos distribuciones se mide utilizando la divergencia de [Kullback-Leibler (KL)](https://es.wikipedia.org/wiki/Divergencia_de_Kullback-Leibler). La divergencia KL representa la cantidad esperada de error generado si aproximamos una distribución con otra. La distribución con parámetros óptimos tendrá la divergencia KL más pequeña cuando se mida con la distribución real. 

Una vez que se ha identificado la distribución óptima, lo que significa que se han identificado los parámetros óptimos, se puede aprovechar para producir las matrices de salida y ejecutar cualquier inferencia requerida.

# Aplicación de LDA
En este Notebook usaremos datos de comentarios realizados por usuarios de Tripadvisor a restaurantes en Bogotá. El objetivo es aplicar la técnica de LDA para extraer algunos tópicos del corpus.

In [1]:
import os 
import pandas as pd

Comenzamos importando la base de datos

In [2]:
os.listdir("data")

['enlaces_tripadvisor.csv',
 'comentarios_tripadvisor.csv',
 '2-simplex.png',
 'LDA1.png',
 'LDA2.png',
 '.ipynb_checkpoints',
 'Scraping_tripadvisor.ipynb']

In [3]:
df = pd.read_csv("data/comentarios_tripadvisor.csv", sep = ";")
df.head()

,nombre,link,puntaje_global,n_comentarios,posicion_relativa,n_restaurantes,lat,lon,excelente,muy_bueno,...,características,usuario,relevancia_usuario,fecha_comentario,titulo_comentario,contenido_comentario,fecha_visita,puntaje,rango_de_precios,dietas_especiales
0,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",Marylyn73,4 opiniones,Escribió una opinión el 6 de enero de 2021,Comida recalentada,Fui temprano para evitar aglomeraciones y me t...,enero de 2021,10.0,NaN,NaN
1,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",jairoenriquer2020,1 opinión,Escribió una opinión el 10 de marzo de 2020,Mala calidad y costo alto,Mala calidad de la carne no es la mamona tradi...,marzo de 2020,10.0,NaN,NaN
2,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",HectorLatorre,55 opiniones,Escribió una opinión el 13 de febrero de 2020,Maravilloso sitio de comida llanera,Este es un restaurante bastante típico y muy b...,enero de 2020,40.0,NaN,NaN
3,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",Ingriofercas,1 opinión,Escribió una opinión el 13 de enero de 2020,Buen Restaurante Tipico!!!,"buen servicio, eramos mas o menos 20 personas ...",diciembre de 2019,50.0,NaN,NaN
4,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",profet2016,429 opiniones,Escribió una opinión el 12 de octubre de 2019,Un recuerdo de la cultura llanera,"Muy buen ambiente, buena comida y muy buena mú...",octubre de 2019,50.0,NaN,NaN


En total tenemos más de 130 mil comentarios (los cuales representan cada una de las filas del dataframe) y 25 variables que describen el restaurante y el comentario.

In [4]:
df.shape 

(132728, 25)

Primero vamos a quedarnos solo con las columnas que nos interesan: `titulo_comentario` y `contenido_comentario`. Luego las unimos.

In [5]:
# Vamos a reducir el tamaño de nuestra muestra para facilitar le computo de
# los procesos
# df = df.sample(10000, random_state = 666).reset_index(drop = True)

In [5]:
comentarios = df["titulo_comentario"] + " " + df["contenido_comentario"]

Procedemos a preprocesar el texto

In [6]:
import unidecode
import re

# Convertimos la columna en texto
comentarios = comentarios.astype(str)

# Quitamos tildes
comentarios = comentarios.apply(lambda x: unidecode.unidecode(x))

# Quitamos comas, guiones y otros caracteres especiales o signos de puntuación
comentarios = comentarios.apply(lambda x:
    re.sub('[^A-Za-z0-9 ]+', ' ', x))

# Ponemos todo el texto en minúscula 
comentarios = comentarios.str.lower()

# Dejamos todos los espacios sencillos
comentarios = comentarios.apply(lambda x: 
    re.sub('\s+', ' ', x))

# Vamos a eliminar todos los números
comentarios = comentarios.apply(lambda x: re.sub("\d+", "", x))
comentarios = comentarios.apply(lambda x: re.sub('\s+', ' ', x))
comentarios = comentarios.str.strip()

In [7]:
# Eliminamos stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words("spanish"))

# Creamos un diccionario de stopwords en español
stopwords = [unidecode.unidecode(i) for i in stopwords]
stopwords = set(stopwords)

# Creamos una función que elimine las palabras presentes en un diccionario
def eliminar_stopwords(texto, diccionario):
    texto = [tok for tok in texto.split(" ") if tok not in diccionario]
    return(texto)

# Aplicamos la función para eliminar los stopwords
comentarios = comentarios.apply(lambda x: 
    eliminar_stopwords(x, stopwords))

In [8]:
# Tokenizamos el texto
import spacy
nlp = spacy.load("es_core_news_sm")

# Esto puede tardar un poco
comentarios = comentarios.apply(lambda x: nlp(" ".join(x)))

In [9]:
# Lemmatizamos el texto
comentarios = comentarios.apply(lambda x: [unidecode.unidecode(tok.lemma_) for tok in x])

In [11]:
# Para aplicar LDA necesitamos construir un diccionario
import gensim.corpora as corpora

# A cada palabra se le asigna un id
diccionario = corpora.Dictionary(comentarios)

diccionario.most_common()[0:10]

[('buen', 86367),
 ('comida', 69954),
 ('excelente', 51889),
 ('lugar', 49806),
 ('servicio', 47345),
 ('restaurante', 44469),
 ('atencion', 35712),
 ('plato', 32503),
 ('mejor', 30294),
 ('ambiente', 29256)]

In [12]:
# Creamos un corpus
textos = comentarios.copy()

In [13]:
# Frequencias 
corpus = [diccionario.doc2bow(texto) for texto in textos]

In [14]:
# Aplicamos LDA
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint

# Debemos escoger el número de tópicos antes de correr el modelo
n_topicos = 3
# Modelo LDA
lda_model = LdaMulticore(corpus = corpus,
    id2word = diccionario,
    num_topics = n_topicos)
# Mostramos las palabras dentro de los 10 tópicos
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.035*"buen" + 0.028*"comida" + 0.025*"excelente" + 0.021*"atencion" + '
  '0.021*"restaurante" + 0.015*"plato" + 0.014*"ambiente" + 0.013*"servicio" + '
  '0.010*"sabor" + 0.009*"agradable"'),
 (1,
  '0.029*"lugar" + 0.029*"buen" + 0.020*"comida" + 0.019*"excelente" + '
  '0.016*"mejor" + 0.015*"servicio" + 0.012*"ambiente" + 0.012*"bogota" + '
  '0.012*"restaurante" + 0.009*"delicioso"'),
 (2,
  '0.023*"buen" + 0.021*"comida" + 0.019*"servicio" + 0.013*"restaurante" + '
  '0.010*"plato" + 0.009*"precio" + 0.009*"atencion" + 0.008*"lugar" + '
  '0.007*"bien" + 0.007*"calidad"')]


#### VISUALIZACION

La salida de los modelos LDA en Python usando sklearn puede ser difícil de interpretar en forma cruda. Como es el caso en la mayoría de los ejercicios de modelado, las visualizaciones pueden ser un gran beneficio cuando se trata de interpretar y comunicar los resultados del modelo. Una librería de Python, pyLDAvis, se integra directamente con el objeto del modelo sklearn para producir gráficos sencillos. Esta herramienta de visualización devuelve un histograma que muestra las palabras más relacionadas con cada tópico y un biplot, de uso frecuente en PCA, donde cada círculo corresponde a un tópico. A partir del biplot, sabemos la prevalencia de cada tópico en todo el corpus, lo que se refleja en el área del círculo, y la similitud de los tópicos, que se refleja en la cercanía de los círculos.


El escenario ideal es que los círculos se extiendan por toda la trama y tengan un tamaño razonable y consistente. Es decir, queremos que los tópicos sean distintos y que aparezcan uniformemente en todo el corpus. Además de los gráficos de pyLDAvis, aprovecharemos el modelo t-SNE, discutido en un capítulo anterior, para producir una representación bidimensional de la matriz tópico-documento, una matriz donde cada fila representa un documento y cada columna representa la probabilidad de ese tópico que describe el documento.

Habiendo completado el ajuste del modelo LDA, creemos algunos gráficos para ayudarnos a profundizar en los resultados.



In [16]:
# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(lda_model, corpus, diccionario)

/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/iggy/opt/anaconda3/lib/python3.9/site-

/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/iggy/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. 

In [17]:
LDA_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.027942  0.028041       1        1  39.022465
2      0.051333  0.001708       2        1  31.723365
0     -0.023391 -0.029749       3        1  29.254170, topic_info=            Term          Freq         Total Category  logprob  loglift
175    excelente  50007.000000  50007.000000  Default  30.0000  30.0000
177        lugar  47524.000000  47524.000000  Default  29.0000  29.0000
80      atencion  34972.000000  34972.000000  Default  28.0000  28.0000
59      ambiente  28162.000000  28162.000000  Default  27.0000  27.0000
55   restaurante  44856.000000  44856.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
177        lugar   6075.096966  47524.933394   Topic3  -4.9786  -0.8279
135        mejor   5022.551129  29923.772294   Topic3  -5.1689  -0.5556
44        comido   4010.827815  19297.037797   Topic3  -5.3938  -0.3418
23         carne   3543.187390  12787.308394   Topic3  -5.5178  -0.0543
207       bogota   3394.081411  19980.941766   Topic3  -5.5608  -0.5436

[324 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
807       1  0.176964  aceptable
807       2  0.742800  aceptable
807       3  0.080028  aceptable
199       1  0.593760   acogedor
199       2  0.102568   acogedor
...     ...       ...        ...
7970      1  0.928339          z
7970      3  0.065147          z
100       1  0.628523       zona
100       2  0.162085       zona
100       3  0.209438       zona

[644 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

Note que puede seleccionar manualmente cada tema para ver sus términos más frecuentes y/o "relevantes", utilizando diferentes valores del parámetro $\lambda$. Esto puede ayudar cuando intenta asignar un nombre interpretable por humanos o un "significado" a cada tema.

Los valores de lambda ($\lambda$) que están muy cerca de cero mostrarán términos que son más específicos para un tema elegido. Lo que significa que verá términos que son "importantes" para ese tema específico pero no necesariamente "importantes" para todo el corpus.

Los valores de lambda que están muy cerca de uno mostrarán aquellos términos que tienen la relación más alta entre la frecuencia de los términos para ese tema específico y la frecuencia general de los términos del corpus.

In [17]:
# Guardamos la visualización como un html. 
# Es mucho más sencillo interactuar con la gráfica desde el archivo que 
# desde el notebook
pyLDAvis.save_html(LDA_visualization, 'visualizacion_LDA.html')

## Consideraciones Finales

Cuando debemos extraer información de una gran colección de documentos aún no vista, el modelado de tópicos es un gran enfoque, ya que proporciona información sobre la estructura subyacente de los documentos. Es decir, los modelos de tópicos encuentran agrupaciones de palabras utilizando la proximidad, no el contexto.

En este cuaderno, aprendimos cómo aplicar dos de los algoritmos de modelado de tópicos más comunes y efectivos: la asignación de Dirichlet latente y la factorización de matriz no negativa. Ahora deberíamos sentirnos cómodos limpiando documentos de texto sin formato utilizando varias técnicas diferentes; técnicas que se pueden utilizar en muchos otros escenarios de modelado. Continuaremos aprendiendo cómo convertir el corpus limpio en la estructura de datos adecuada de recuentos de palabras sin procesar o pesos de palabras por documento mediante la aplicación de modelos de bolsa de palabras.

El enfoque principal del cuaderno fue ajustar los dos modelos de tópicos, incluida la optimización de la cantidad de tópicos, la conversión de la salida en tablas fáciles de interpretar y la visualización de los resultados. Con esta información, deberíamos poder aplicar modelos de tópicos completamente funcionales para obtener valor e información para cualquier negocio.



Los modelos de tópicos se pueden usar para predecir los tópicos que pertenecen a documentos no vistos, pero si vamos a hacer predicciones, es importante reconocer que los modelos de tópicos sólo conocen las palabras que se usan para entrenarlos. Es decir, si los documentos no vistos tienen palabras que no estaban en los datos de entrenamiento, el modelo no podrá procesar esas palabras incluso si se vinculan a uno de los tópico identificados en los datos de entrenamiento. Debido a este hecho, los modelos de tópicos tienden a usarse más para el análisis exploratorio y la inferencia que para la predicción.

Cada modelo de tópicos genera dos matrices. La primera matriz contiene palabras contra tópicos. Esta enumera cada palabra relacionada con cada tópico con alguna cuantificación de la relación. Dada la cantidad de palabras que considera el modelo, cada tópico sólo se describirá con una cantidad relativamente pequeña de palabras.

Las palabras se pueden asignar a un tópico o a varios tópicos con diferentes cuantificaciones. Si las palabras se asignan a uno o varios tópicos depende del algoritmo. De manera similar, la segunda matriz contiene documentos contra tópicos. Esta asigna cada documento a cada tópico mediante alguna cuantificación de la relación de cada combinación de tópico del documento.

Cuando se analiza el modelado de tópicos, es importante reforzar continuamente el hecho de que los grupos de palabras que representan los tópicos no están relacionados conceptualmente; están relacionados solo por proximidad. La proximidad frecuente de ciertas palabras en los documentos es suficiente para definir tópicos debido a una suposición establecida anteriormente: que todas las palabras en el mismo documento están relacionadas.

Sin embargo, esta suposición puede no ser cierta o las palabras pueden ser demasiado genéricas para formar tópicos coherentes. La interpretación de tópicos abstractos implica equilibrar las características innatas de los datos de texto con las agrupaciones de palabras generadas. Los datos de texto, y el lenguaje en general, son muy variables, complejos y contextuales, lo que significa que cualquier resultado generalizado debe consumirse con cautela.

Esto no es para minimizar o invalidar los resultados del modelo. Dados documentos cuidadosamente limpios y una cantidad adecuada de tópicos, las agrupaciones de palabras, como veremos, pueden ser una buena guía sobre lo que contiene un corpus y pueden incorporarse de manera efectiva en sistemas de datos más grandes.

Ya discutimos algunas de las limitaciones de los modelos de tópicos, pero hay algunos puntos adicionales que debemos considerar. La naturaleza ruidosa de los datos de texto puede hacer que los modelos de tópico asignen palabras no relacionadas con uno de los tópicos a ese tópico en particular.

Nuevamente, consideremos la oración sobre el trabajo de antes. La palabra reunión podría aparecer en la agrupación de palabras que representa el tópico de trabajo. También es posible que la palabra larga pueda estar en ese grupo, pero la palabra larga no está directamente relacionada con el trabajo. Larga puede estar en el grupo porque aparece con frecuencia muy cerca de la palabra reunión. Por lo tanto, larga probablemente se consideraría falsamente (o espuriamente) correlacionado con el trabajo y probablemente debería eliminarse de la agrupación de tópicos, si es posible. Las palabras falsamente correlacionadas en grupos de palabras pueden causar problemas significativos cuando analizando los datos.

Esto no es necesariamente una falla en el modelo. En cambio, es una característica que, dados datos ruidosos, el modelo podría extraer peculiaridades de los datos que podrían afectar negativamente los resultados. Las correlaciones espurias podrían ser el resultado de cómo, dónde o cuándo se recopilaron los datos. Si los documentos se recopilaron sólo en una región geográfica específica, las palabras asociadas con esa región podrían vincularse incorrectamente, aunque accidentalmente, a una o varias de las agrupaciones de palabras resultantes del modelo.

Tenga en cuenta que, con palabras adicionales en el grupo de palabras, podríamos adjuntar más documentos a ese tópico de los que deberían adjuntarse. Si reducimos la cantidad de palabras que pertenecen a un tópico, ese tópico se asignará a menos documentos. Tenga en cuenta que esto no es algo malo. Queremos que cada grupo de palabras contenga solo palabras que tengan sentido para que podamos asignar los tópicos apropiados a los documentos apropiados.

## Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Blei, D. M., Jordan, M. I., &; Ng, A. Y. (2003). Latent Dirichlet Allocation. JMLR.org, 3, 993–1022. https://doi.org/10.5555/944919.944937 

- Patel, A. A. (2019). Hands-on unsupervised learning using Python: how to build applied machine learning solutions from unlabeled data. O'Reilly Media.

- Murphy, K. P. (2012). Machine learning: a probabilistic perspective. MIT press.